In [1]:
import dask.array as ds
import dask
import os
from satpy import Scene, MultiScene, find_files_and_readers
from satpy.composites import GenericCompositor
from satpy.writers import to_image
from trollimage.colormap import spectral
from pyresample.geometry import AreaDefinition
import yaml
import cartopy
from glob import glob
import xarray as xr

ERROR 1: PROJ: proj_create_from_database: Open of /home/janky/miniforge3/envs/ALGOTLvisTest/share/proj failed


In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt

In [ ]:
from dask.distributed import Client
client = Client()  # start distributed scheduler locally.
client

In [8]:
from satpy.utils import check_satpy
check_satpy()

Readers
abi_l1b:  ok
abi_l1b_scmi:  ok
abi_l2_nc:  ok
acspo:  ok
agri_fy4a_l1:  ok
agri_fy4b_l1:  ok
ahi_hrit:  ok
ahi_hsd:  ok
ahi_l1b_gridded_bin:  ok
ahi_l2_nc:  ok
ami_l1b:  ok
amsr2_l1b:  ok
amsr2_l2:  ok
amsr2_l2_gaasp:  ok
amsub_l1c_aapp:  ok
ascat_l2_soilmoisture_bufr:  cannot find module 'satpy.readers.ascat_l2_soilmoisture_bufr' (('Missing eccodes-python and/or eccodes C-library installation. Use conda to install eccodes.\n           Error: ', ModuleNotFoundError("No module named 'eccodes'")))
atms_l1b_nc:  ok
atms_sdr_hdf5:  ok
avhrr_l1b_aapp:  ok
avhrr_l1b_eps:  ok
avhrr_l1b_gaclac:  cannot find module 'satpy.readers.avhrr_l1b_gaclac' (No module named 'pygac')
avhrr_l1b_hrpt:  ok
avhrr_l1c_eum_gac_fdr_nc:  ok
caliop_l2_cloud:  cannot find module 'satpy.readers.caliop_l2_cloud' (No module named 'pyhdf')
clavrx:  cannot find module 'satpy.readers.clavrx' (No module named 'pyhdf')
cmsaf-claas2_l2_nc:  ok
electrol_hrit:  ok
epic_l1b_h5:  ok
fci_l1c_nc:  ok
fci_l2_nc:  ok
generi

In [5]:
from satpy.utils import debug_on
debug_on()

In [3]:
os.environ['SATPY_DASK_CHUNK_SIZE'] = '32MiB'
dask.config.set(num_workers=4)

In [4]:
# TODO Modularize
with open('../../config/satvis.yaml', 'r') as f:
    config = yaml.safe_load(f)

dir_dict = config['data_directories']
base_path = dir_dict["sentinel2_data"]

In [ ]:
18.67500 57.43500

In [9]:
algotl_10 = AreaDefinition.from_area_of_interest(area_id="algotltest_full", projection='epsg:3006', shape=[20000, 18000], center=[720573.556, 6371774.759], resolution=[10, 10])
# algotl_20 = AreaDefinition.from_area_of_interest(area_id="algotltest_full", projection='epsg:4326', shape=[5000, 4500], lat_0= center=[57.43500, 18.67500], resolution=[40, 40])

In [ ]:
algotl_20

In [5]:
# 20000x18000, center=[720573.556, 6371774.759]
c_x = 720573.556
c_y = 6371774.759
ll_x = 630573.556
ll_y = 6271774.759
ur_x = 810573.556
ur_y = 6471774.759

c = Point(c_x, c_y)
ll = Point((ll_x, ll_y))
ul = Point((ll_x, ur_y))
lr = Point((ur_x, ll_y))
ur = Point((ur_x, ur_y))
# print('GEOMETRYCOLLECTION(' + str(ll) + ',', str(ul) + ',', str(lr) + ',', str(ur) + ')')

In [6]:
overlap = 0.1
quad_width = 10000
quad_height = 9000
ol_x = quad_width * overlap
ol_y = quad_height * overlap
quad_x_ol = quad_width + ol_x
quad_y_ol = quad_height + ol_y

# p_ll_c = Point((ll_x+(c_x+ol))/2, (ll_y+(c_y+ol))/2)
# p_ul_c = Point((ll_x+(c_x+ol))/2, (ur_y+(c_y-ol))/2)
# p_lr_c = Point((ur_x+(c_x-ol))/2, (ll_y+(c_y+ol))/2)
# p_ur_c = Point((ur_x+(c_x-ol))/2, (ur_y+(c_y-ol))/2)
# print('GEOMETRYCOLLECTION(' + str(ll_c) + ',', str(ul_c) + ',', str(lr_c) + ',', str(ur_c) + ')')
# GEOMETRYCOLLECTION(POINT (675573.556 6321774.759), POINT (675573.556 6421774.759), POINT (765573.556 6321774.759), POINT (765573.556 6421774.759))
# GEOMETRYCOLLECTION(POINT (676073.556 6322274.759), POINT (676073.556 6421274.759), POIN1T (765073.556 6322274.759), POINT (765073.556 6421274.759))


ll_c = [(ll_x+(c_x+ol_x))/2, (ll_y+(c_y+ol_y))/2]
ul_c = [(ll_x+(c_x+ol_x))/2, (ur_y+(c_y-ol_y))/2]
lr_c = [(ur_x+(c_x-ol_x))/2, (ll_y+(c_y+ol_y))/2]
ur_c = [(ur_x+(c_x-ol_x))/2, (ur_y+(c_y-ol_y))/2]

center_points = {'ll': ll_c, 'ul': ul_c, 'lr': lr_c, 'ur': ur_c}

In [7]:
algotl_quadrants = []
for quadrant, center in center_points.items():
    algotl_bbox = AreaDefinition.from_area_of_interest(area_id=f"algotl_{quadrant}", projection='epsg:3006', shape=[quad_x_ol, quad_y_ol], center=center, resolution=[10, 10])
    algotl_quadrants.append(algotl_bbox)

In [11]:
algotl_bbox.crs

<Projected CRS: EPSG:3006>
Name: SWEREF99 TM
Axis Info [cartesian]:
- N[north]: Northing (metre)
- E[east]: Easting (metre)
Area of Use:
- name: Sweden - onshore and offshore.
- bounds: (10.03, 54.96, 24.17, 69.07)
Coordinate Operation:
- name: SWEREF99 TM
- method: Transverse Mercator
Datum: SWEREF99
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [8]:
datequad_path = '/220627/ul'
dlist = [os.path.join(base_path + datequad_path, dir) for dir in os.listdir(base_path + datequad_path)]

In [9]:
dlist

['/home/janky/Data/ALGOTL/220627/ul/6',
 '/home/janky/Data/ALGOTL/220627/ul/5',
 '/home/janky/Data/ALGOTL/220627/ul/3',
 '/home/janky/Data/ALGOTL/220627/ul/2']

In [10]:
scn_list = []
for path in dlist:
    files = find_files_and_readers(base_dir=path, reader='msi_safe')
    scn = Scene(filenames=files, reader='msi_safe')
    scn_list.append(scn)
scn_list

In [ ]:
algotl_quadrants

In [11]:
mscn = MultiScene(scn_list)
mscn.load(['B02', 'B03', 'B04'])
if datequad_path == '/220627/ll':
    algotl_mscn = mscn.resample(algotl_quadrants[0])
elif datequad_path == '/220627/ul':
    algotl_mscn = mscn.resample(algotl_quadrants[1])
elif datequad_path == '/220627/lr':
    algotl_mscn = mscn.resample(algotl_quadrants[2])
elif datequad_path == '/220627/ur':
    algotl_mscn = mscn.resample(algotl_quadrants[3])
else:
    algotl_mscn = mscn.resample(algotl_quadrants[0])
algotl_scn = algotl_mscn.blend()

/home/janky/miniforge3/envs/ALGOTLvisTest/lib/python3.11/site-packages/pyresample/kd_tree.py:1153: PerformanceWarning: Increasing number of chunks by factor of 11
  res = blockwise(_my_index, dst_adims,
/home/janky/miniforge3/envs/ALGOTLvisTest/lib/python3.11/site-packages/pyresample/kd_tree.py:1153: PerformanceWarning: Increasing number of chunks by factor of 11
  res = blockwise(_my_index, dst_adims,
/home/janky/miniforge3/envs/ALGOTLvisTest/lib/python3.11/site-packages/pyresample/kd_tree.py:1153: PerformanceWarning: Increasing number of chunks by factor of 11
  res = blockwise(_my_index, dst_adims,
/home/janky/miniforge3/envs/ALGOTLvisTest/lib/python3.11/site-packages/pyresample/kd_tree.py:1153: PerformanceWarning: Increasing number of chunks by factor of 17
  res = blockwise(_my_index, dst_adims,
/home/janky/miniforge3/envs/ALGOTLvisTest/lib/python3.11/site-packages/pyresample/kd_tree.py:1153: PerformanceWarning: Increasing number of chunks by factor of 17
  res = blockwise(_my_ind

In [11]:
print(algotl_scn)

<xarray.DataArray 'my_index-8735d939ff8fe68b00ee96991d592eff' (y: 20000,
                                                               x: 18000)> Size: 1GB
dask.array<where, shape=(20000, 18000), dtype=float32, chunksize=(4096, 4096), chunktype=numpy.ndarray>
Coordinates:
    band         int64 8B 1
    spatial_ref  int64 8B 0
    crs          object 8B PROJCRS["SWEREF99 TM",BASEGEOGCRS["SWEREF99",DATUM...
  * y            (y) float64 160kB 6.472e+06 6.472e+06 ... 6.272e+06 6.272e+06
  * x            (x) float64 144kB 6.306e+05 6.306e+05 ... 8.106e+05 8.106e+05
Attributes: (12/16)
    wavelength:           0.49 µm (0.44-0.54 µm)
    _satpy_id:            DataID(name='B02', wavelength=WavelengthRange(min=0...
    reader:               msi_safe
    standard_name:        toa_bidirectional_reflectance
    resolution:           10
    platform_name:        Sentinel-2A
    ...                   ...
    end_time:             2022-06-27 10:06:11
    start_time:           2022-06-27 10:06:11
 

In [16]:
algotl_cropped = algotl_scn.crop(area=algotl_10)
algotl_resampled = algotl_cropped.resample(algotl_10)

DEBUG:pyresample.geometry:Projections for data and slice areas are identical: PROJCRS["unknown",BASEGEOGCRS["unknown",DATUM["Unknown based on WGS 84 ellipsoid",ELLIPSOID["WGS 84",6378137,298.257223563,LENGTHUNIT["metre",1],ID["EPSG",7030]]],PRIMEM["Greenwich",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8901]]],CONVERSION["unknown",METHOD["Mercator (variant B)",ID["EPSG",9805]],PARAMETER["Latitude of 1st standard parallel",0,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8823]],PARAMETER["Longitude of natural origin",15,ANGLEUNIT["degree",0.0174532925199433],ID["EPSG",8802]],PARAMETER["False easting",0,LENGTHUNIT["metre",1],ID["EPSG",8806]],PARAMETER["False northing",0,LENGTHUNIT["metre",1],ID["EPSG",8807]]],CS[Cartesian,2],AXIS["(E)",east,ORDER[1],LENGTHUNIT["metre",1,ID["EPSG",9001]]],AXIS["(N)",north,ORDER[2],LENGTHUNIT["metre",1,ID["EPSG",9001]]]]
[DEBUG: 2024-05-10 16:26:09 : pyresample.geometry] Projections for data and slice areas are identical: PROJCRS["unknown",BASEGEO

In [ ]:
algotl_scn_20 = algotl_scn.aggregate(func='mean', x=10, y=10)

In [12]:
algotl_scn.save_datasets(writer='cf', datasets=['B02', 'B03', 'B04'], filename='SEN2_lr_220627.nc')

[None]

In [23]:
algotl_scn.save_datasets(writer='geotiff')

[DEBUG: 2024-05-10 17:31:38 : satpy.writers] Reading ['/home/janky/miniforge3/envs/ALGOTLvisTest/lib/python3.11/site-packages/satpy/etc/writers/geotiff.yaml']


[DEBUG: 2024-05-10 17:31:38 : satpy.writers] Adding enhancement configuration from file: /home/janky/miniforge3/envs/ALGOTLvisTest/lib/python3.11/site-packages/satpy/etc/enhancements/generic.yaml
[DEBUG: 2024-05-10 17:31:38 : satpy.writers] Data for DataID(name='B02', wavelength=WavelengthRange(min=0.44, central=0.49, max=0.54, unit='µm'), resolution=10, calibration=<1>, modifiers=()) will be enhanced with options:
	[{'name': 'linear_stretch', 'method': <function stretch at 0x7f99fc7a4ea0>, 'kwargs': {'stretch': 'crude', 'min_stretch': 0.0, 'max_stretch': 100.0}}, {'name': 'gamma', 'method': <function gamma at 0x7f99fc7a4f40>, 'kwargs': {'gamma': 1.5}}]
[DEBUG: 2024-05-10 17:31:38 : trollimage.xrimage] Applying stretch crude with parameters {'min_stretch': 0.0, 'max_stretch': 100.0}
[DEBUG: 2024-05-10 17:31:38 : trollimage.xrimage] Applying gamma 1.5
[DEBUG: 2024-05-10 17:31:38 : rasterio.env] Entering env context: <rasterio.env.Env object at 0x7f99fc5d0750>
[DEBUG: 2024-05-10 17:31:38

TypeError: ('Could not serialize object of type HighLevelGraph', '<ToPickle: HighLevelGraph with 5 layers.\n<dask.highlevelgraph.HighLevelGraph object at 0x7f99fc6198d0>\n 0. store-sources-47608f4af1cc6899e51b5124cd9f5f10\n 1. store-map-6f1470a0efe191fc0a3e70d30e165bf4\n 2. store-map-5e2594f3a2d95eddb3831ffee0ac4932\n 3. store-map-6de1696c74d49e0322671a0c3de568c5\n 4. store-c6e9dafcc7e07c6c073ea482d2a5d50d\n>')

In [ ]:
b04min = algotl_scn['B04'].values.min()
b04max = algotl_scn['B04'].values.max()
b03min = algotl_scn['B03'].values.min()
b03max = algotl_scn['B03'].values.max()
b02min = algotl_scn['B02'].values.min()
b02max = algotl_scn['B02'].values.max()

In [ ]:
algotl_scn['B04_stretch'] = (algotl_scn['B04'] - b04min * 2) / (b04max - b04min)
# algotl_scn['B04_stretch'].attrs = combine_metadata(algotl_scn['B04'])

algotl_scn['B03_stretch'] = (algotl_scn['B03'] - b03min * 2) / (b03max - b03min)
# algotl_scn['B03_stretch'].attrs = combine_metadata(algotl_scn['B03'])

algotl_scn['B02_stretch'] = (algotl_scn['B02'] - b02min * 2) / (b02max - b02min)
# algotl_scn['B02_stretch'].attrs = combine_metadata(algotl_scn['B02'])

In [24]:
compositor = GenericCompositor('overview')

composite = compositor([algotl_scn['B04'],
                        algotl_scn['B03'],
                        algotl_scn['B02']])

In [25]:
img = to_image(composite)

In [26]:
# img.invert([False, False, True])
# img.stretch('crude', max_stretch=10)
img.stretch('linear', cutoffs=(0.4, 0.4))

In [27]:
img.gamma(1.7)

In [28]:
img.show()

In [ ]:
mins = 1
maxs = 1.45

img = to_image(algotl_scn['kahru2'])
img.stretch('crude', min_stretch=[mins], max_stretch=[maxs])
spectral.set_range(mins, maxs)
# spectral.set_range(1, 2)
img.colorize(spectral)
img.show()

In [ ]:
algotl_scn.save_datasets(writer='simple_image', filename='algotl_diff_test_output.png', base_dir='/home/janky/Data/')